In [1]:
import glob
import os
import re
import anal_games
import functions_anal

In [8]:


# Ensure the output directory exists
output_dir = "../Cleaned_Analyzed_Games"
os.makedirs(output_dir, exist_ok=True)

# Get the list of filenames matching the patterns
filenames_15 = glob.glob("../Analyzed_Games/twic*_15_analyzed.csv")
filenames_16 = glob.glob("../Analyzed_Games/twic*_16_analyzed.csv")

# Create dictionaries to map numbers to filenames
filenames_15_dict = {}
for filename in filenames_15:
    basename = os.path.basename(filename)
    match = re.match(r'twic(\d+)_15_analyzed\.csv', basename)
    if match:
        number = match.group(1)
        filenames_15_dict[number] = filename

filenames_16_dict = {}
for filename in filenames_16:
    basename = os.path.basename(filename)
    match = re.match(r'twic(\d+)_16_analyzed\.csv', basename)
    if match:
        number = match.group(1)
        filenames_16_dict[number] = filename

# Combine the numbers and prioritize '_16' files
all_numbers = set(filenames_15_dict.keys()).union(set(filenames_16_dict.keys()))
filenames_to_process = []

for number in sorted(all_numbers):
    if number in filenames_16_dict:
        # If both versions exist, keep only the '_16' version
        filenames_to_process.append(filenames_16_dict[number])
    elif number in filenames_15_dict:
        filenames_to_process.append(filenames_15_dict[number])

# Process each file individually
for file in filenames_to_process:
    # Get the base filename without the directory
    base_filename = os.path.basename(file)

    # Determine the output filename in the new directory
    output_file = os.path.join(output_dir, base_filename.replace("_analyzed.csv", "_processed.csv"))

    # Check if the output file already exists; skip processing if it does
    if os.path.isfile(output_file):
        print(f"Output file {output_file} already exists. Skipping {file}.")
        continue

    print(f"Processing {file}...")

    # Process the file using process_one_file
    df = anal_games.process_one_file(
        filename=file,
        functions=[functions_anal.Cleanup],  # Include any other functions you wish to apply
        game_wise=False  # Set to False if you want to retain move data
    )

    # Save the processed DataFrame to the output CSV file
    df.to_csv(output_file, index=False)

    print(f"Saved processed data to {output_file}")


Processing ../Analyzed_Games\twic1201_16_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic1201_16_processed.csv
Processing ../Analyzed_Games\twic1202_16_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic1202_16_processed.csv
Processing ../Analyzed_Games\twic1203_16_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic1203_16_processed.csv
Processing ../Analyzed_Games\twic1204_16_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic1204_16_processed.csv
Processing ../Analyzed_Games\twic1205_16_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic1205_16_processed.csv
Processing ../Analyzed_Games\twic1206_16_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic1206_16_processed.csv
Processing ../Analyzed_Games\twic1207_16_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic1207_16_processed.csv
Processing ../Analyzed_Games\twic1208_16_analyzed.csv...
Saved proces

c:\Users\foivo\Documents\chess project\data_analysis\anal_games.py:145: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(filename)


Saved processed data to ../Cleaned_Analyzed_Games\twic1549_15_processed.csv
Processing ../Analyzed_Games\twic1556_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic1556_15_processed.csv
Processing ../Analyzed_Games\twic1557_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic1557_15_processed.csv
Processing ../Analyzed_Games\twic1558_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic1558_15_processed.csv
Processing ../Analyzed_Games\twic1559_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic1559_15_processed.csv
Processing ../Analyzed_Games\twic1560_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic1560_15_processed.csv
Processing ../Analyzed_Games\twic920_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic920_15_processed.csv
Processing ../Analyzed_Games\twic921_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic921_15_processed.c

c:\Users\foivo\Documents\chess project\data_analysis\anal_games.py:145: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(filename)


Saved processed data to ../Cleaned_Analyzed_Games\twic923_15_processed.csv
Processing ../Analyzed_Games\twic924_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic924_15_processed.csv
Processing ../Analyzed_Games\twic935_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic935_15_processed.csv
Processing ../Analyzed_Games\twic936_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic936_15_processed.csv
Processing ../Analyzed_Games\twic937_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic937_15_processed.csv
Processing ../Analyzed_Games\twic938_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic938_15_processed.csv
Processing ../Analyzed_Games\twic939_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic939_15_processed.csv
Processing ../Analyzed_Games\twic940_15_analyzed.csv...


c:\Users\foivo\Documents\chess project\data_analysis\anal_games.py:145: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(filename)


Saved processed data to ../Cleaned_Analyzed_Games\twic940_15_processed.csv
Processing ../Analyzed_Games\twic941_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic941_15_processed.csv
Processing ../Analyzed_Games\twic942_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic942_15_processed.csv
Processing ../Analyzed_Games\twic943_15_analyzed.csv...


c:\Users\foivo\Documents\chess project\data_analysis\anal_games.py:145: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(filename)


Saved processed data to ../Cleaned_Analyzed_Games\twic943_15_processed.csv
Processing ../Analyzed_Games\twic944_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic944_15_processed.csv
Processing ../Analyzed_Games\twic945_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic945_15_processed.csv
Processing ../Analyzed_Games\twic946_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic946_15_processed.csv
Processing ../Analyzed_Games\twic947_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic947_15_processed.csv
Processing ../Analyzed_Games\twic948_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic948_15_processed.csv
Processing ../Analyzed_Games\twic949_15_analyzed.csv...
Saved processed data to ../Cleaned_Analyzed_Games\twic949_15_processed.csv
